# ESTIMATION MAXIMIZATION ALGORITHM

### Predicting coin flips



Implementation steps :

1. Extract the data : 30 iterations for both the coins - 20 flips per coin
2. Process the data to a list and calculate heads for each iteration
3. Randomize bias for each coin
4. Perform E-Step : estimation for expected number of heads and tails for each coin across the trials
5. Perform M-Step : using the estimates, we recompute better bias (update Theta)
6. Repeat steps 4,5 until Theta values converge (do not change)


In [5]:
import urllib, json
import ast
import numpy as np

In [7]:
url = "https://24zl01u3ff.execute-api.us-west-1.amazonaws.com/beta" #input url

coin_flips_heads = [] # initializing the list that should contain fetched data : heads/tails for each flip
for i in range(1,31): # fetching 30 times starting with index 1 to 30.
    #Make a request to the url, and return the status code
    req = urllib.request.Request(url) # fetch the data using Request function on url object(which contains the url string)

    ##parsing response
    r = urllib.request.urlopen(req).read() #reading the fetched data saved in req object to 
    cont = json.loads(r.decode('utf-8')) #converting data to json data
    coin_flips=(ast.literal_eval(cont['body'])) # convert the string of list to list object
    # coin_flips contain list of lists : each list within the complete list is a set of 20 flips 
    coin_flips_heads.append(np.array(coin_flips).sum()) #computing sum of all 1s such that it is the head counts.

theta_values = { "ThetaA":.37, "ThetaB":.09 } # initializing random theta values that will later be updated at M step
print("coin flips- counts of heads in each flip:",coin_flips_heads)


while True: ## iterating until the theta values converge
    heads = { "ThetaA":0, "ThetaB":0 } # initializing heads variable for A, B
    tails = { "ThetaA":0, "ThetaB":0 } # initializing tails variable for A, B
    # E-step: calculate probability distributions over possible completions
    for flip in coin_flips_heads: # iterate for each count of heads (tails is 20 - heads)
        # computing binomial probability  i.e., likelihood of a sequence given a weight
        P_E_Z = sum((pow( k, flip ) * pow( 1-k, (20 - flip) )) for k in theta_values.values())
        for key in theta_values.keys():
            p = theta_values[key]
            P_E_a_or_b = pow( p, flip ) * pow( 1-p, (20 - flip) )
            
            weightedProbability = P_E_a_or_b / P_E_Z 
            heads[key] += flip* weightedProbability
            tails[key] += (20 - flip) * weightedProbability

    P_Z_E = sum( theta_values.values())
    # M-step: update values for parameters given current distribution
    for key in theta_values.keys():
        theta_values[key] = heads[key] / (heads[key] + tails[key])

    print( theta_values)
    ### convergence criteria : absolute difference between new theta and old theta (bias) for each coin
    if abs( sum( theta_values.values()) - P_Z_E ) < .00001: 
        break
    

coin flips- counts of heads in each flip: [14, 16, 13, 14, 5, 6, 14, 5, 18, 18, 10, 14, 15, 15, 13, 13, 7, 15, 14, 8, 18, 4, 11, 4, 14, 7, 16, 11, 16, 4]
{'ThetaA': 0.6144693821897249, 'ThetaB': 0.2110837691519024}
{'ThetaA': 0.7068005967873895, 'ThetaB': 0.2692014546157036}
{'ThetaA': 0.7235974444082802, 'ThetaB': 0.29104247415294077}
{'ThetaA': 0.7279785376934923, 'ThetaB': 0.30093718853285034}
{'ThetaA': 0.7298055887931746, 'ThetaB': 0.3052110743141553}
{'ThetaA': 0.730609824532819, 'ThetaB': 0.30709509023425857}
{'ThetaA': 0.7309679003521965, 'ThetaB': 0.3079337532628381}
{'ThetaA': 0.731127971142741, 'ThetaB': 0.30830860198818244}
{'ThetaA': 0.7311996463607362, 'ThetaB': 0.30847643578939526}
{'ThetaA': 0.731231763603283, 'ThetaB': 0.30855163846477857}
{'ThetaA': 0.7312461597269437, 'ThetaB': 0.3085853465180014}
{'ThetaA': 0.7312526135109119, 'ThetaB': 0.30860045773382644}
{'ThetaA': 0.7312555069263625, 'ThetaB': 0.30860723250110783}
